In [28]:
from my_libs_py3 import *
pd.set_option("display.max_rows",100)
import bson

In [2]:
import cudf

In [4]:
cu_data = cudf.from_pandas(data)

NameError: name 'data' is not defined

In [11]:
cu_data.sample(1)

AttributeError: 'DataFrame' object has no attribute 'sample'

In [3]:
df = cudf.DataFrame({"a":{1, 2, 3, 4, 5}})

In [9]:
data = data.drop("_id",axis=1)

In [7]:
data["Refresh_Date"] = data["Refresh_Date"].apply(lambda x: x.strftime("%Y-%m-%d"))

Connection Successful


-----

### My own backtester - pair 

- need to feed a strategy function (maybe just one strategy one backtest)

In [7]:
back_day = 360
window = 20
method = "self_minute"
cash = 20000
market_price=True
exec_next_bar = False
i = "DEN"
j = "CMO"



robinhood = robingateway()

price1 = get_price_data([i], method=method,robinhood=robinhood,back_day=back_day )
price1 = price1.set_index("TimeStamp")
# price1.loc[price1.Open == 0,"Open"] = np.NaN


price2 = get_price_data([j], method=method,robinhood=robinhood,back_day=back_day )
price2 = price2.set_index("TimeStamp")


# set to the same length
if len(price2) != len(price1):
    price1 = get_price_data([i], method="realtimeday",robinhood=robinhood,back_day=back_day )
    price1 = price1.set_index("TimeStamp")
    # price1.loc[price1.Open == 0,"Open"] = np.NaN


    price2 = get_price_data([j], method="realtimeday",robinhood=robinhood,back_day=back_day )
    price2 = price2.set_index("TimeStamp")


price_table = pd.DataFrame()
#--------------------------------------------------------------
#     return price1, price2


price1.loc[:,"log_ret"] = log(price1.Close / price1.Close.shift(1))
price2.loc[:,"log_ret"] = log(price1.Close / price2.Close.shift(1))


price1.loc[:, "log_ret_mv"] = price1.log_ret.rolling(window).mean()
price2.loc[:, "log_ret_mv"] = price2.log_ret.rolling(window).mean()

price1 = price1.fillna(method="bfill")
price2 = price2.fillna(method="bfill")

price_table.loc[:,"relative"] = price1.log_ret_mv / price2.log_ret_mv

price_table.loc[:,"relative_mv"] = price_table["relative"].rolling(window, min_periods=window).mean()
price_table["relative_mv"] = abs(price_table["relative_mv"])

price_table = price_table.join(price1["adjClose"]).rename({"adjClose":"stock1"},axis=1)
price_table = price_table.join(price2["adjClose"]).rename({"adjClose":"stock2"},axis=1)

price_table = price_table.dropna(subset=["stock1","stock2"])

for i in range(1,len(price_table)):
    j = i- window
    if j < 0 :
        j = 0 
    price_table.loc[price_table.iloc[i].name,"slope"] = stats.linregress(price1.log_ret.iloc[j:i],price2.log_ret.iloc[j:i])[0] 
    price_table.loc[price_table.iloc[i].name,"hedge_ratio"]= \
    sm.OLS(price_table.stock2.iloc[j:i],price_table.stock1.iloc[j:i]).fit().params[0]

price1.loc[:,"volatility"] = price1.log_ret.rolling(window).std()
price2.loc[:,"volatility"] = price2.log_ret.rolling(window).std()

price_table["z_score"] =( price_table["relative"]-price_table["relative_mv"])/price_table.relative.std()
# up, mid , low = ta.BBANDS(price1.Close*price_table.hedge_ratio)
up, mid , low = ta.BBANDS(price_table["z_score"])

bb = pd.DataFrame({"up":up,"mid":mid,"low":low})

price_table = price_table.join(bb)
price_table = price_table.fillna(method="bfill")

def buy_line(slope):
    if slope>0.5:
        return -1.25
    elif slope>0.75:
        return -1.75
    elif slope:
        return -2.25
    elif slope<-0.75:
        return -2.75
    else:
        return -2
def sell_line(slope):
    if slope>0.5:
        return 1.25
    elif slope>0.75:
        return 1.75
    elif slope:
        return 2.25
    elif slope<-0.75:
        return 2.75
    else:
        return 2

price_table["buy_line"] = price_table.slope.apply(buy_line)
price_table["sell_line"] = price_table.slope.apply(sell_line)

price_table["cash"] = cash
price_table["buying_power"] = cash
price_table["margin1"] = 0
price_table["margin2"] = 0
price_table["size1"] = 0
price_table["size2"] = 0
price_table["holddays"] = 0

All price data of Close is actually Adj Close
Connection Successful
DEN
('Finished', 'DEN')
All price data of Close is actually Adj Close
Connection Successful
CMO
('Finished', 'CMO')
All price data of Close is actually Adj Close
('Finished', 'DEN')
All price data of Close is actually Adj Close
('Finished', 'CMO')


C:\Users\gli26\Anaconda2\envs\python3\lib\site-packages\scipy\stats\_stats_mstats_common.py:107: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\Users\gli26\Anaconda2\envs\python3\lib\site-packages\scipy\stats\_stats_mstats_common.py:117: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\Users\gli26\Anaconda2\envs\python3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\gli26\Anaconda2\envs\python3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\gli26\Anaconda2\envs\python3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\gli26\Anaconda2\envs\python3\lib\site-packag

In [8]:
def trade(size_col,trade_size):
    '''
    This function is only used in pair trade back test class/function.
    Contians global variables.
    '''
    
    global price_table
    global inx_exe
    global market_price
    global tmp
    
    if "1" in size_col:
        stock_col = "stock1"
        margin_col = "margin1"
    if "2" in size_col:
        stock_col = "stock2"
        margin_col = "margin2"
        
    
    price = price_table.loc[inx_exe,stock_col]
    trade_value = trade_size * price
    
    
    def update():
        price_table.loc[inx_exe,"cash"] -=  trade_value
        price_table.loc[inx_exe,size_col] += trade_size
        price_table.loc[inx_exe,"holddays"] = 0 

    
    
    if market_price:
        if price_table.loc[inx_exe,"cash"] > abs(trade_value):
            update()
            print("%s, Trade Executed, traded %s: %s, at %s, cash flow: %s"%(inx_exe,stock_col,trade_size,price,-trade_value))
    else:
        if price_table.loc[inx_exe,"cash"] > abs(trade_value) and price<= tmp[stock_col]:
            update()
            print("%s, Trade Executed, traded %s: %s, at %s, cash flow: %s"%(inx_exe,stock_col,trade_size,price,-trade_value))
        
    
    ##margin
    if price_table.loc[inx_exe,size_col]<0:
        print("adjusted cash for margin")
        price_table.loc[inx_exe,"cash"] -= abs(trade_value)
        price_table.loc[inx_exe,margin_col] = abs(trade_value)
        
    

def close(size_col):
    '''
    This function is only used in pair trade back test class/function.
    Contians global variables.
    '''
    print("Close")
    global price_table
    global inx_exe
    global market_price
    global tmp
    
    if "1" in size_col:
        stock_col = "stock1"
        margin_col = "margin1"
    if "2" in size_col:
        stock_col = "stock2"
        margin_col = "margin2"
        
    
    price = price_table.loc[inx_exe,stock_col]
    trade_size = (0-price_table.loc[inx_exe,size_col])
    trade_value =  trade_size* price
    
    
    if price_table.loc[inx_exe,size_col]<0:
        price_table.loc[inx_exe,"cash"] += price_table.loc[inx_exe,margin_col]
        price_table.loc[inx_exe,margin_col] = 0
   
    
  
    price_table.loc[inx_exe,"cash"] -=  trade_value

    price_table.loc[inx_exe,size_col] = 0

    price_table.loc[inx_exe,"holddays"] = 0 
    
    print("%s, Trade Executed, traded %s: %s, at %s, cash flow: %s"%(inx_exe,stock_col,trade_size,price,-trade_value))
    
    

    

for iday in range(len(price_table.index)):
    ## inherite the trade properties
    if iday>0:
        inx = price_table.index[iday]
        inx_pre = price_table.index[iday-1]
#         price_table.loc[inx,"holddays"] =price_table.loc[inx_pre,"holddays"]
        price_table.loc[inx,"size1"] =price_table.loc[inx_pre,"size1"]
        price_table.loc[inx,"size2"]= price_table.loc[inx_pre,"size2"]
        price_table.loc[inx,"cash"] =price_table.loc[inx_pre,"cash"]
        price_table.loc[inx,"margin1"] =price_table.loc[inx_pre,"margin1"]
        price_table.loc[inx,"margin2"] =price_table.loc[inx_pre,"margin2"]
        price_table.loc[inx,"buying_power"] = price_table.loc[inx,"cash"]+ \
                                price_table.loc[inx,"margin1"] -price_table.loc[inx,"margin2"] 
    else:
        inx = price_table.index[iday]
    
    
    ## adjust holddays
    if price_table.loc[inx,"size1"] != 0 or price_table.loc[inx,"size2"] != 0:
        price_table.loc[inx,"holddays"] =  price_table.loc[inx_pre,"holddays"] +1
    
    ### use tmp to represent this trade bar
    tmp = price_table.iloc[iday]
    
    if exec_next_bar:
        ## eday is used to write values, iday is used to read values
        inx_exe = price_table.index[iday+1]
        if eday == len(price_table): continue
    else:
        inx_exe = price_table.index[iday]
    
    ## Action each day -- Check long buy signals
    if tmp.z_score < tmp.buy_line and tmp.size1 <= 0:
        
        if tmp.size1 !=0:
            close("size1")
            tmp = price_table.iloc[iday]
        if tmp.size2 !=0:
            close("size2")
            tmp = price_table.iloc[iday]
        

        # control the size to be long only if wish
        new_size1 = int(tmp.cash/(tmp.stock1 + tmp.stock1* tmp.hedge_ratio))
        new_size2 = int(-new_size1 * tmp.hedge_ratio)
        trade1_size = new_size1-tmp.size1
        trade2_size = new_size2-tmp.size2
        
        if abs(trade1_size)*tmp.stock1 + abs(trade2_size)*tmp.stock2 <= tmp.cash:
            trade("size1",trade1_size)
            trade("size2",trade2_size)
        
        
    ## Action each day -- Check short buy signals
    elif tmp.z_score > tmp.sell_line and tmp.size1 >= 0:
        if tmp.size1 !=0:
            close("size1")
            tmp = price_table.iloc[iday]
        if tmp.size2 !=0:
            close("size2")
            tmp = price_table.iloc[iday]
        

        # control the size to be long only if wish
        new_size1 = - int(tmp.cash/(tmp.stock1 + tmp.stock1* tmp.hedge_ratio))
        new_size2 = int(-new_size1 * tmp.hedge_ratio)
        trade1_size = new_size1-tmp.size1
        trade2_size = new_size2-tmp.size2
        
        ## trade
        if abs(trade1_size)*tmp.stock1 + abs(trade2_size)*tmp.stock2 <= tmp.cash:
            trade("size1",trade1_size)
            trade("size2",trade2_size)
        
    ## Action each day -- Close trade
    elif tmp.holddays>=15:
        close("size1")
        close("size2")
        
price_table["Net_Values"] = price_table.cash + price_table.size1*price_table.stock1 + \
                        price_table.size2*price_table.stock2 + price_table.margin1+price_table.margin2

price_table["return"] = (price_table.Net_Values-price_table.Net_Values.shift(1))/price_table.Net_Values

2020-10-20 00:00:00, Trade Executed, traded stock1: 905.0, at 16.889999, cash flow: -15285.449095
2020-10-20 00:00:00, Trade Executed, traded stock2: -278.0, at 5.316021, cash flow: 1477.853838
adjusted cash for margin
Close
2020-11-10 00:00:00, Trade Executed, traded stock1: -905.0, at 18.219999, cash flow: 16489.099095
Close
2020-11-10 00:00:00, Trade Executed, traded stock2: 278.0, at 5.401609, cash flow: -1501.6473019999999


In [9]:
price_table

,relative,relative_mv,stock1,stock2,slope,hedge_ratio,z_score,up,mid,low,buy_line,sell_line,cash,buying_power,margin1,margin2,size1,size2,holddays,Net_Values,return
TimeStamp,,,,,,,,,,,,,,,,,,,,,
2020-09-21,-0.0034,0.0034,18.1000,5.6096,0.0000,0.3099,-1.3252,-1.0405,-1.6688,-2.2970,-2.0000,2.0000,"20,000.0000","20,000.0000",0,0.0000,0.0000,0.0000,0,"20,000.0000",nan
2020-09-22,-0.0034,0.0034,19.8500,5.4520,0.0000,0.3099,-1.3252,-1.0405,-1.6688,-2.2970,-2.0000,2.0000,"20,000.0000","20,000.0000",0,0.0000,0.0000,0.0000,0,"20,000.0000",0.0000
2020-09-23,-0.0034,0.0034,19.2700,5.2388,0.0000,0.2907,-1.3252,-1.0405,-1.6688,-2.2970,-2.0000,2.0000,"20,000.0000","20,000.0000",0,0.0000,0.0000,0.0000,0,"20,000.0000",0.0000
2020-09-24,-0.0034,0.0034,18.8900,5.2573,0.0000,0.2843,-1.3252,-1.0405,-1.6688,-2.2970,-2.0000,2.0000,"20,000.0000","20,000.0000",0,0.0000,0.0000,0.0000,0,"20,000.0000",0.0000
2020-09-25,-0.0034,0.0034,18.7400,5.3593,0.0000,0.2828,-1.3252,-1.0405,-1.6688,-2.2970,-2.0000,2.0000,"20,000.0000","20,000.0000",0,0.0000,0.0000,0.0000,0,"20,000.0000",0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-05,0.0045,0.0034,55.4100,6.5300,0.7597,0.1273,0.2337,0.3202,-0.0519,-0.4240,-1.2500,1.2500,"21,179.8565","21,179.8565",0,0.0000,0.0000,0.0000,0,"21,179.8565",0.0000
2021-05-06,0.0054,0.0035,57.0000,6.4800,0.3335,0.1263,0.3691,0.5346,0.0442,-0.4463,-2.2500,2.2500,"21,179.8565","21,179.8565",0,0.0000,0.0000,0.0000,0,"21,179.8565",0.0000
2021-05-07,0.0058,0.0037,58.8100,6.4900,0.3208,0.1249,0.4051,0.5605,0.1940,-0.1725,-2.2500,2.2500,"21,179.8565","21,179.8565",0,0.0000,0.0000,0.0000,0,"21,179.8565",0.0000


In [10]:
price_table["return"].mean()

0.0003489074006842845

In [11]:
price_table["return"].std()

0.004288755869870445

In [12]:
price_table["Net_Values"].iloc[-1]

21179.856536

In [13]:
SR = price_table["return"].mean()/price_table["return"].std()

In [14]:
SR

0.0813539896582699

In [15]:
TR = (price_table["Net_Values"].iloc[-1] - price_table["Net_Values"].iloc[0])/price_table["Net_Values"].iloc[0]

In [16]:
TR

0.05899282679999997

In [2]:
self = robingateway()

In [ ]:
== "Trade Success!":
                log_trade(self.short_symbol,-log["size"].sum(), realtimequote(self.short_symbol).price.values[0], "vix_dayroll")
                send_email(body_html="",body_content="", title = "VIX Dayroll exit Signal")

In [9]:
robinhood = robingateway()





########################   
# Fix unsetteled trade
########################

try:

    log_pos = get_open_opsition()
    temp = robinhood.get_my_positions()
    pos_frame = pd.DataFrame([temp[0],temp[1]]).transpose()
    pos_frame.columns = ["Ticker","Shares"]
    #     act_pos = robinhood.get_my_positions()[0]
    note = ""
    for i in log_pos:
        if i not in pos_frame.Ticker.to_list():
    #             note += ("fix "+ i +"\n")
            fix_unsettled_trade_update(i)
            send_email("Fixed unsettled trade to size 0 " + str(i))
        elif get_trade_log(i)["size"].iloc[0] != pos_frame.loc[pos_frame.Ticker == i,"Shares"].iloc[0]:
            fix_unsettled_trade_update(i, size=pos_frame.loc[pos_frame.Ticker == i,"Shares"].iloc[0], partial=True)
            send_email("Fixed unsettled trade to match size " + str(i))

except Exception as e:
    send_email("Fixed unsettled trade Fail\n" + str(e))


Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection